Load libraries and set parameters

In [1]:
from PIL import Image, ImageFont, ImageDraw
import os
import random
import os, shutil
import numpy as np

#Funtoin to clear a directory of all files
def delete_files_in_directory(folder):
   for filename in os.listdir(folder):
    file_path = os.path.join(folder, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print('Failed to delete %s. Reason: %s' % (file_path, e))

#Function to take target shape and turn it into a numerical class for labelling
def shape_to_class(shape):
    switcher = {
        "circle": 0,
        "cross": 1,
        "pentagon": 2,
        "quartercircle": 3,
        "rectangle": 4,
        "semicircle": 5,
        "star": 6,
        "triangle": 7
    }
    return switcher.get(shape,"nothing")

#Define possible characteristics for targets
White = (255, 255, 255)
Black = (0, 0, 0)
Red = (255, 0, 0)
Blue = (0, 0, 255)
Green = (0, 255, 0)
Purple = (128, 0, 128)
Brown = (150, 75, 0)
Orange = (255, 165, 0)
colors = [White, Black, Red, Blue, Green, Purple, Brown, Orange]
strcolors = ['White', 'Black', 'Red', 'Blue',
             'Green', 'Purple', 'Brown', 'Orange']
letters = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M", "N", "O", "P", "Q",
    "R", "S", "T", "U", "V", "W", "X", "Y", "Z", "0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]
shapes = []
backgrounds = []

Preload Shape Plates

In [2]:
for Sfiles in os.listdir('Shape Plate'):
    if Sfiles.endswith('.png'):
        shapes.append(Sfiles)

Preload Backgrounds

In [3]:
for Bfiles in os.listdir('background'):
    if Bfiles.endswith('.png'):
        backgrounds.append(Bfiles)

Load Font and Font Size

In [4]:
alphanumSize = 70
myfont = ImageFont.truetype('calibrib.ttf', alphanumSize)

Set Image Parameters and Set Amount of Images to Create

In [5]:
buffer = 138
shapeReHeight = 105
shapeReWidth = 136
amount = 500

Clear Old Images

In [6]:
delete_files_in_directory("final/images")
delete_files_in_directory("final/labels")

Code To Generate Images

In [7]:
for i in range(amount):
    background = random.choice(backgrounds)
    Background = Image.open(f'background/{background}')
    width, height = Background.size

    #Location of target in Q1
    xpos1 = random.randint(0, width/2-buffer)
    ypos1 = random.randint(0, height/2-buffer)
    #Location of target in Q2
    xpos2 = random.randint(width/2, width-buffer)
    ypos2 = random.randint(0, height/2-buffer)
    #Location of target in Q3
    xpos3 = random.randint(0, width/2-buffer)
    ypos3 = random.randint(height/2, height-buffer)
    #Location of target in Q4
    xpos4 = random.randint(width/2, width-buffer)
    ypos4 = random.randint(height/2, height-buffer)
    
    xpos = [xpos1,xpos2,xpos3,xpos4]
    ypos = [ypos1,ypos2,ypos3,ypos4]

    for tNum in range(4):
        print("Image#:",i," Target#:",tNum)

        #Selects random characteristics for targets
        letter = random.choice(letters)
        shape = random.choice(shapes)
        choice1 = random.randint(0, len(colors)-1)
        choice2 = random.randint(0, len(colors)-1)
        while (choice1 == choice2):
            choice2 = random.randint(0, len(colors)-1)
        shapeColor = colors[choice1]
        shapeColorSTR = strcolors[choice1]
        alphaColor = colors[choice2]
        alphaColorSTR = strcolors[choice2]
        

        Shape = Image.open(f'Shape Plate/{shape}')
        Sshape, fext = os.path.splitext(shape)

        #Shape = Shape.resize((int(shapeReWidth), int(shapeReHeight)))
        shapeWidth, shapeHeight = Shape.size
        
        im = Shape.convert('RGBA')

        # "data" is a height x width x 4 numpy array
        data = np.array(im)
        red, green, blue, alpha = data.T  # Temporarily unpack the bands for readability

        # Replace white with red... (leaves alpha values alone...)
        white_areas = (red == 255) & (blue == 255) & (green == 255)

        # Transpose back needed
        data[..., :-1][white_areas.T] = (shapeColor)

        ColoredShape = Image.fromarray(data)
        #ColoredShape = ColoredShape.filter(ImageFilter.BLUR)

        target = ImageDraw.Draw(ColoredShape)
        
        if Sshape == 'circle': #Circle
            target.text((54,60), letter, font=myfont, fill=alphaColor, anchor='mm', align='center')
        elif Sshape == 'cross': #Cross
            target.text((54,55), letter, font=myfont, fill=alphaColor, anchor='mm', align='center')
        elif Sshape == 'pentagon': #Pentagon
            target.text((52,61), letter, font=myfont, fill=alphaColor, anchor='mm', align='center')
        elif Sshape == 'quartercircle': #Quarter Circle
            target.text((40,60), letter, font=myfont, fill=alphaColor, anchor='mm', align='center')
        elif Sshape == 'rectangle': #Rectangle
            target.text((65,51), letter, font=myfont, fill=alphaColor, anchor='mm', align='center')
        elif Sshape == 'semicircle': #Semicircle
            target.text((68,40), letter, font=myfont, fill=alphaColor, anchor='mm', align='center')
        elif Sshape == 'star':#Star
            target.text((60,62), letter, font=myfont, fill=alphaColor, anchor='mm', align='center')
        elif Sshape == 'triangle':#Triangle
            target.text((68,75), letter, font=myfont, fill=alphaColor, anchor='mm', align='center')
        else:
            target.text((0,0), letter, font=myfont, fill=alphaColor, anchor='mm', align='center')

        Background.paste(ColoredShape, (xpos[tNum], ypos[tNum]), ColoredShape)

        file_name = str(i).zfill(5)
    
        with open('final/labels/{}.txt'.format(file_name), 'a', encoding='utf-8') as f:
            f.write('{} {} {} {} {}\n'.format(shape_to_class(Sshape),
                                            (xpos[tNum]+(shapeWidth/2))/width,
                                            (ypos[tNum]+(shapeHeight/2))/height,
                                            (shapeWidth)/width,
                                            (shapeHeight)/height))
    #blurImage = Background.filter(ImageFilter.BLUR)
    #blurImage = blurImage.filter(ImageFilter.BLUR)
    #blurImage.save('final/images/{}.jpg'.format(file_name))
    Background.save(f'final/images/{file_name}.jpg')

Image#: 0  Target#: 0
Image#: 0  Target#: 1
Image#: 0  Target#: 2
Image#: 0  Target#: 3
Image#: 1  Target#: 0
Image#: 1  Target#: 1
Image#: 1  Target#: 2
Image#: 1  Target#: 3
Image#: 2  Target#: 0
Image#: 2  Target#: 1
Image#: 2  Target#: 2
Image#: 2  Target#: 3
Image#: 3  Target#: 0
Image#: 3  Target#: 1
Image#: 3  Target#: 2
Image#: 3  Target#: 3
Image#: 4  Target#: 0
Image#: 4  Target#: 1
Image#: 4  Target#: 2
Image#: 4  Target#: 3
Image#: 5  Target#: 0
Image#: 5  Target#: 1
Image#: 5  Target#: 2
Image#: 5  Target#: 3
Image#: 6  Target#: 0
Image#: 6  Target#: 1
Image#: 6  Target#: 2
Image#: 6  Target#: 3
Image#: 7  Target#: 0
Image#: 7  Target#: 1
Image#: 7  Target#: 2
Image#: 7  Target#: 3
Image#: 8  Target#: 0
Image#: 8  Target#: 1
Image#: 8  Target#: 2
Image#: 8  Target#: 3
Image#: 9  Target#: 0
Image#: 9  Target#: 1
Image#: 9  Target#: 2
Image#: 9  Target#: 3
Image#: 10  Target#: 0
Image#: 10  Target#: 1
Image#: 10  Target#: 2
Image#: 10  Target#: 3
Image#: 11  Target#: 0
Image